In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('SecurityData.csv', low_memory=False)
print(data.head())

   GVKEY LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  datadate    tic  \
0   3263       NR      NaN      NaN  11/30/77  10/31/17  10/31/17   CWLO   
1   5965       NU      NaN      NaN    5/3/03         E   2/28/17   INTO   
2   1140       NU      NaN      NaN  10/28/86   1/31/06   1/31/06  5022B   
3   3584       NR      NaN      NaN  11/14/09         E   8/31/22   APYI   
4   1681       NR      NaN      NaN   10/9/87   9/30/19   5/31/98   ASOE   

                          conm isalrt   cshtrm curcdm  navm  prccm  prchm  \
0       COMMONWEALTH OIL REFIN    NaN    947.0    USD   NaN   9.00   9.00   
1                   INITIO INC    NaN  19000.0    USD   NaN   0.25   0.25   
2  ADVANCED COMMUNICATIONS INC    NaN   4000.0    USD   NaN   2.58   2.58   
3                   ASPYRA INC    NaN  12021.0    USD   NaN   0.05   0.05   
4      APOLLO SOLAR ENERGY INC    NaN  55400.0    USD   NaN   3.00   3.00   

    prclm         trt1m secstat  fic ccmbegdt  
0  9.0000  5.882400e+00       I 

In [2]:
# Filter the data
df = data[['GVKEY', 'datadate', 'trt1m', 'prccm']].copy()
print(df.head())

   GVKEY  datadate         trt1m  prccm
0   3263  10/31/17  5.882400e+00   9.00
1   5965   2/28/17  2.499990e+07   0.25
2   1140   1/31/06           NaN   2.58
3   3584   8/31/22  4.999900e+06   0.05
4   1681   5/31/98  2.999900e+06   3.00


In [3]:
# Convert the date column to datetime
df['date'] = pd.to_datetime(df['datadate'], format='%m/%d/%y', errors='coerce')

# Assuming all future dates should actually be in the past, adjust the century
df['date'] = df['date'].apply(lambda x: x if x.year <= pd.Timestamp.now().year else x - pd.DateOffset(years=100))
df.drop('datadate', axis=1, inplace=True)
df = df.rename(columns={'trt1m': 'return','prccm': 'close'})

print(df.head())

   GVKEY        return  close       date
0   3263  5.882400e+00   9.00 2017-10-31
1   5965  2.499990e+07   0.25 2017-02-28
2   1140           NaN   2.58 2006-01-31
3   3584  4.999900e+06   0.05 2022-08-31
4   1681  2.999900e+06   3.00 1998-05-31


In [4]:
# Sort the returns
df.sort_values(by=['GVKEY', 'date'], inplace=True)

# Check if the close prices on a given day and in the previous month are smaller than 1
# If so, let the return be NaN
df['prev_close'] = df.groupby('GVKEY')['close'].shift(1)

# Exclude the abnormal data (growth_rate != return)
df['growth_rate'] = (df['close'] - df['prev_close']) / df['prev_close'] * 100
df['diff'] = abs(df['return'] - df['growth_rate'])

df.loc[(df['prev_close'] <= 1), 'return'] = np.nan

print(df.head())

         GVKEY   return  close       date  prev_close  growth_rate  \
1022685   1000      NaN  11.75 1970-09-30         NaN          NaN   
258082    1000   6.3830  12.50 1970-10-31       11.75     6.382979   
971376    1000 -28.0000   9.00 1970-11-30       12.50   -28.000000   
162604    1000  11.1111  10.00 1970-12-31        9.00    11.111111   
842662    1000 -10.0000   9.00 1971-01-31       10.00   -10.000000   

                 diff  
1022685           NaN  
258082   2.127660e-05  
971376   3.552714e-15  
162604   1.111111e-05  
842662   0.000000e+00  


In [5]:
df = df.iloc[:, :-3]
# Calculate Momentum for the Prior (2-12) Month Period using the filtered data
df.sort_values(by=['GVKEY', 'date'], inplace=True)
df['momentum'] = df['return'].transform(lambda x: (x.rolling(window=11, min_periods=11).apply(lambda y: ((y / 100) + 1).prod())).shift(2))

# Ensure there are no NaN values in your data
df.dropna(subset=['momentum'], inplace=True)

# Calculate Decile Breakpoints and Assign Stocks to Deciles using the filtered data
df['decile'] = df.groupby('date')['momentum'].transform(lambda x: pd.qcut(x, 10, labels=range(1, 11)))
 
print(df.head())

        GVKEY   return  close       date  momentum decile
284978   1000   5.4054  4.875 1971-10-31  0.446809      1
910835   1000 -15.3846  4.125 1971-11-30  0.370000      1
24938    1000  39.3939  5.750 1971-12-31  0.541666      1
556263   1000   0.0000  5.750 1972-01-31  0.412500      1
817053   1000  -8.6957  5.250 1972-02-29  0.638888      1


In [6]:
# Calculate the average return for each decile on each date and round to two decimal places
aggregated_returns = df.groupby([df['date'].dt.strftime('%Y%m'), 'decile'], observed=True)['return'].mean().reset_index()
aggregated_returns['return'] = aggregated_returns['return'].round(2)

# Pivot the Data
pivot_table = aggregated_returns.pivot(index='date', columns='decile', values='return')

# Rename Columns for Output
column_names = {
    1: 'Lo PRIOR',
    10: 'Hi PRIOR'
}
column_names.update({i: f'PRIOR {i}' for i in range(2, 10)})
pivot_table.columns = [column_names.get(x, x) for x in pivot_table.columns]

# Print the formatted DataFrame with two decimal places
print(pivot_table)

        Lo PRIOR  PRIOR 2  PRIOR 3  PRIOR 4  PRIOR 5  PRIOR 6  PRIOR 7  \
date                                                                     
196206     -8.45    -8.53    -7.35    -9.06    -8.74    -7.34    -5.56   
196207      4.36     7.77     5.60     5.34     8.05     6.42     8.75   
196208      2.50     1.79     2.79     4.32     1.62     3.08     4.41   
196209     -4.44    -7.05    -4.35    -8.35    -6.36    -2.53    -2.83   
196210     -0.94    -1.51    -2.91    -2.78    -0.46    -3.08    -2.34   
...          ...      ...      ...      ...      ...      ...      ...   
202310     -4.50    -4.06    -5.15    -2.58    -5.26    -3.91    -4.60   
202311      8.71     7.91     7.04     8.90     8.21     6.07     6.71   
202312     11.46     8.85     6.76     5.55     7.02     6.02     4.88   
202401     -1.31    -1.42    -3.05    -0.32    -1.41     0.70    -1.45   
202402       NaN      NaN      NaN      NaN      NaN      NaN      NaN   

        PRIOR 8  PRIOR 9  Hi PRIOR  


In [7]:
pivot_table.to_csv('replicated_portfolio.csv', index=True, header=True)

In [8]:
fama = pd.read_csv('10_Portfolios_Prior_12_2.csv',skiprows=1178,nrows=1164)
fama['Unnamed: 0'] = pd.to_datetime(fama['Unnamed: 0'], format='%Y%m')
fama.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
print(fama)

           Date  Lo PRIOR  PRIOR 2  PRIOR 3  PRIOR 4  PRIOR 5  PRIOR 6  \
0    1927-01-01     -0.47     1.04     4.17     2.40    -0.31     3.08   
1    1927-02-01      8.49     3.88     8.33     7.59     4.79     5.47   
2    1927-03-01     -6.15    -5.11    -2.38    -1.60    -1.63    -1.70   
3    1927-04-01      2.61    -0.93     0.10    -1.20     0.09     0.40   
4    1927-05-01      1.12     3.73     6.01     5.17     7.55     9.27   
...         ...       ...      ...      ...      ...      ...      ...   
1159 2023-08-01    -12.08    -6.40    -5.72    -5.52    -3.85    -2.21   
1160 2023-09-01    -12.11    -6.39    -6.78    -3.65    -3.35    -3.77   
1161 2023-10-01    -11.82    -8.04    -5.47    -4.49    -5.69    -4.99   
1162 2023-11-01      7.80     8.71     7.19     8.15     7.74     5.38   
1163 2023-12-01     15.16    13.28    14.01    11.92    10.53     9.33   

      PRIOR 7  PRIOR 8  PRIOR 9  Hi PRIOR  
0        1.48     0.64    -0.59      1.18  
1        5.40     4.30 

In [9]:
estimation = pd.read_csv('replicated_portfolio.csv')
estimation['date'] = pd.to_datetime(estimation['date'], format='%Y%m')
estimation.rename(columns={'date': 'Date'}, inplace=True)
print(estimation)

          Date  Lo PRIOR  PRIOR 2  PRIOR 3  PRIOR 4  PRIOR 5  PRIOR 6  \
0   1962-06-01     -8.45    -8.53    -7.35    -9.06    -8.74    -7.34   
1   1962-07-01      4.36     7.77     5.60     5.34     8.05     6.42   
2   1962-08-01      2.50     1.79     2.79     4.32     1.62     3.08   
3   1962-09-01     -4.44    -7.05    -4.35    -8.35    -6.36    -2.53   
4   1962-10-01     -0.94    -1.51    -2.91    -2.78    -0.46    -3.08   
..         ...       ...      ...      ...      ...      ...      ...   
736 2023-10-01     -4.50    -4.06    -5.15    -2.58    -5.26    -3.91   
737 2023-11-01      8.71     7.91     7.04     8.90     8.21     6.07   
738 2023-12-01     11.46     8.85     6.76     5.55     7.02     6.02   
739 2024-01-01     -1.31    -1.42    -3.05    -0.32    -1.41     0.70   
740 2024-02-01       NaN      NaN      NaN      NaN      NaN      NaN   

     PRIOR 7  PRIOR 8  PRIOR 9  Hi PRIOR  
0      -5.56    -5.83    -7.82    -10.88  
1       8.75     9.77     6.66      4

In [10]:
# Merge the two dataframes based on 'Date' column
merged_df = pd.merge(fama, estimation, on='Date', suffixes=('_fama', '_estimation'))

# Remove rows with missing values
merged_df.dropna(inplace=True)

# Calculate correlation between corresponding columns
correlation_matrix = merged_df.corr()

#Filter correlation values for desired columns
correlation_values = correlation_matrix.loc['Lo PRIOR_fama':'Hi PRIOR_fama', 'Lo PRIOR_estimation':'Hi PRIOR_estimation']

print(correlation_values)

diagonal_correlation = correlation_values.values.diagonal()
print("Correlation between corresponding columns:")
print(diagonal_correlation)

               Lo PRIOR_estimation  PRIOR 2_estimation  PRIOR 3_estimation  \
Lo PRIOR_fama             0.905793            0.859230            0.820893   
PRIOR 2_fama              0.925836            0.944899            0.930214   
PRIOR 3_fama              0.912450            0.952180            0.954771   
PRIOR 4_fama              0.892144            0.938708            0.950240   
PRIOR 5_fama              0.877822            0.928663            0.942137   
PRIOR 6_fama              0.848168            0.900298            0.922248   
PRIOR 7_fama              0.821848            0.871554            0.900909   
PRIOR 8_fama              0.806830            0.846696            0.871943   
PRIOR 9_fama              0.768592            0.801163            0.830510   
Hi PRIOR_fama             0.718819            0.729652            0.751010   

               PRIOR 4_estimation  PRIOR 5_estimation  PRIOR 6_estimation  \
Lo PRIOR_fama            0.787245            0.760519           